In [5]:
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By
import numpy as np

In [304]:
def Crawling(driver, page, type, url_xpath, df_alchol, i):
    url = 'https://business.veluga.kr/search/result/?page=' + str(page) + '&drink_type_id=' + str(type)
    driver.get(url)
    click = driver.find_element(By.XPATH, url_xpath)
    driver.execute_script("arguments[0].click();", click)
    time.sleep(3)
    image_element = driver.find_elements(By.XPATH,'//*[@id="__next"]/div/div/div[1]/img')[0]
    image_td = image_element.get_attribute('src')

    name =  driver.find_element(By.XPATH,'//*[@id="__next"]/div/div/div[2]/div[1]/h1').text
    
    percent =  driver.find_element(By.XPATH,"//th[contains(text(), '도수')]")
    percent_td = percent.find_element(By.XPATH,"following-sibling::td").text

    category =  driver.find_element(By.XPATH,"//th[contains(text(), '스타일')]")
    category_td = category.find_element(By.XPATH,"following-sibling::td").text

    produce =  driver.find_element(By.XPATH,"//th[contains(text(), '제조사')]")
    produce_td = produce.find_element(By.XPATH,"following-sibling::td").text

    supply = driver.find_element(By.XPATH,"//th[contains(text(), '공급사')]")
    supply_td = supply.find_element(By.XPATH,"following-sibling::td").text

    index = index + 1

    df_alchol.loc[index] = [name, percent_td, category_td, produce_td, supply_td, image_td]
    i+1
    pass

In [ ]:
from selenium.common.exceptions import ElementNotInteractableException, ElementClickInterceptedException, NoSuchElementException

df_alchol = pd.DataFrame(columns = ['name', 'percent', 'category', 'produce', 'supply', 'img_url'])
driver = webdriver.Chrome(executable_path='chromedriver')
index = 0

for page in range(1, 3+1):
    ## 크롤링할 페이지 변수 
    type = 7
    url = 'https://business.veluga.kr/search/result/?page=' + str(page) + '&drink_type_id=' + str(type)
    driver.get(url)

    print("현재 " + str(page) + "페이지 크롤링 중")
 
    # 페이지의 총 20개 아이템을 하나씩 돌면서 크롤링
    for i in range(1, 20+1):
        
        time.sleep(3)
        base_xpath = '//*[@id="__next"]/div/div/div[1]/a['
        tail_xpath = ']/div[1]/img'
        url_xpath = base_xpath + str(i) + tail_xpath
        
        try: 
            click = driver.find_element(By.XPATH, url_xpath)
            driver.execute_script("arguments[0].click();", click)
            time.sleep(3)

            # 이미지 url 
            image_element = driver.find_elements(By.XPATH,'//*[@id="__next"]/div/div/div[1]/img')[0]
            image_td = image_element.get_attribute('src')

            # 주류 이름
            name =  driver.find_element(By.XPATH,'//*[@id="__next"]/div/div/div[2]/div[1]/h1').text
            
            # 주류 도수
            percent =  driver.find_element(By.XPATH,"//th[contains(text(), '도수')]")
            percent_td = percent.find_element(By.XPATH,"following-sibling::td").text

            # 주류 카테고리
            category =  driver.find_element(By.XPATH,"//th[contains(text(), '스타일')]")
            category_td = category.find_element(By.XPATH,"following-sibling::td").text

            # 주류 제조사
            produce =  driver.find_element(By.XPATH,"//th[contains(text(), '제조사')]")
            produce_td = produce.find_element(By.XPATH,"following-sibling::td").text
            # 주류 공급사
            supply = driver.find_element(By.XPATH,"//th[contains(text(), '공급사')]")
            supply_td = supply.find_element(By.XPATH,"following-sibling::td").text

            # 크롤링한 데이터 데이터프레임에 넣기
            df_alchol.loc[index] = [name, percent_td, category_td, produce_td, supply_td, image_td] 

            # 데이터 프레임 인덱스 증가
            index = index + 1

        # 오류가 발생했을 경우 발생한 페이지 재접속하여 크롤링 시도
        except ElementNotInteractableException:
            print(category_td + " pass")
            Crawling(driver, page, type, url_xpath, df_alchol, i)


        except ElementClickInterceptedException:
            print(category_td + " pass")
            Crawling(driver, page, type, url_xpath, df_alchol, i)


        except NoSuchElementException:
            print( category_td + " NoSuchElementException")
            Crawling(driver, page, type, url_xpath, df_alchol, i)


        except IndexError:
            print( category_td + " IndexError")
            Crawling(driver, page, type, url_xpath, df_alchol, i)


        driver.back()   


In [307]:
df_alchol.head(10)

,name,percent,category,produce,supply,img_url
0,경주법주 쌀막걸리,6.0%,탁주,금복주,경주법주 주식회사,https://cdn.veluga.kr/drinks/0/main/a6c27b6144...
1,가와지탁주 9.5,9.5%,탁주 막걸리,배다리도가,배다리도가,https://cdn.veluga.kr/files/supplier/205/drink...
2,한탄강익스프레스,12.0%,탁주 막걸리,농업회사법인(주)우창 두루미양조장,농업회사법인주식회사우창,https://cdn.veluga.kr/files/supplier/256/drink...
3,본,17.0%,탁주 막걸리,농업회사법인(주)우창 두루미양조장,농업회사법인주식회사우창,https://cdn.veluga.kr/files/supplier/256/drink...
4,대관람차,12.0%,탁주 막걸리,농업회사법인(주)우창 두루미양조장,농업회사법인주식회사우창,https://cdn.veluga.kr/files/supplier/256/drink...
5,신선주 탁주10%,10.0%,탁주 막걸리,농업회사법인 (주)신선,농업회사법인 주식회사 신선,https://cdn.veluga.kr/files/supplier/255/drink...
6,가와지탁주 7.5,7.5%,탁주 막걸리,배다리도가,배다리도가,https://cdn.veluga.kr/files/supplier/205/drink...
7,우리쌀 배다리막걸리,7.0%,탁주,고양탁주합동제조장,배다리도가,https://cdn.veluga.kr/files/supplier/205/drink...
8,구절초꽃술 11도,11.0%,탁주,농업회사법인한통술이노베이션주식회사,농업회사법인한통술이노베이션주식회사,https://assets.business.veluga.kr/media/public...
9,구절초꽃술 8도,8.0%,탁주,농업회사법인한통술이노베이션주식회사,농업회사법인한통술이노베이션주식회사,https://assets.business.veluga.kr/media/public...


In [334]:
alchol_df1 = pd.read_csv('alchol_all.csv')
alchol_df1.drop(columns= "Unnamed: 0", inplace= True)
alchol_df2 = pd.read_csv('alchol_crawling_df1', index_col=0)
alchol_df3 = pd.read_csv('alchol_crawling_df2', index_col=0)
alchol_df4 = pd.read_csv('alchol_crawling_df3', index_col=0)

In [4]:
alchol_df = pd.concat([alchol_df1, alchol_df2,alchol_df3,alchol_df4], ignore_index= True)

NameError: name 'pd' is not defined

In [346]:
alchol_df.to_csv('alchol_df.csv')

In [9]:
alchol_df = pd.read_csv('data/alchol_df.csv', index_col= 0)

In [10]:
alchol_df

,name,percent,category,produce,supply,img_url,580
0,국향,13.0%,청주,롯데칠성음료,롯데칠성음료 국내부문,https://cdn.veluga.kr/files/supplier/155/drink...,NaN
1,설화,14.0%,청주,롯데칠성음료,롯데칠성음료 국내부문,https://cdn.veluga.kr/files/supplier/155/drink...,NaN
2,청하,13.0%,청주,롯데칠성음료,롯데칠성음료 국내부문,https://cdn.veluga.kr/files/supplier/155/drink...,NaN
3,청하 드라이,13.5%,청주,롯데칠성음료,롯데칠성음료 국내부문,https://cdn.veluga.kr/files/supplier/155/drink...,NaN
4,백화수복,13.0%,청주,롯데칠성음료,롯데칠성음료 국내부문,https://cdn.veluga.kr/files/supplier/155/drink...,NaN
...,...,...,...,...,...,...,...
1329,오희 스파클링 막걸리,8.5%,탁주,문경주조,부국상사,https://assets.business.veluga.kr/media/public...,NaN
1330,붉은 원숭이,10.8%,탁주,농업회사법인 (주)술샘,부국상사,https://assets.business.veluga.kr/media/public...,NaN
1331,술취한 원숭이,10.8%,탁주,농업회사법인 (주)술샘,부국상사,https://assets.business.veluga.kr/media/public...,NaN
1332,호땅,6.0%,탁주 막걸리,배혜정도가,한국술유통,https://cdn.veluga.kr/files/supplier/temporary...,NaN


In [46]:
alchol_df['category'].unique()

array([nan, '스피릿', '칵테일', '증류식', '사이더', '맥주', '기타', '윗비어', '페일 라거', '라거',
       '스타우트', '사워', '임페리얼 스타우트', '와일드 에일', 'IPA', '필스너', '더블 IPA',
       '페일 에일', '헬레스 라거', '고제', '메르첸', '벨지안 에일', '블론드 에일', '트리펠', '람빅',
       '프루트 비어', '바이젠 복', '바이젠', '위트 비어', '두벨', '세종', '포터', '발틱 포터',
       '벨지안 스트롱 에일', '임페리얼 IPA', '브라운 에일', '베를리너 바이세', '다크 IPA', '세션 IPA',
       '위트 에일', '다크 라거', '골든 에일', '엠버 에일', '쿼드루펠', '라들러', '괴즈',
       '잉글리시 스트롱 에일', '헤페 바이젠', '프루티드 람빅', '올드 에일', '쾰쉬', '아메리칸 스트롱 에일',
       '발리 와인', '아이스 복', '도펠 복', '둥켈', '엠버 라거', '그리셋', '스모크 비어', '둥켈 바이젠',
       '스카치 에일', '희석식', '막걸리'], dtype=object)

In [21]:
alchol_df['class'] = np.nan
alchol_df['category1'] = np.nan

category = alchol_df['category'].str.split(' ')

for i, j in enumerate(category):
    
    if len(j) == 1 :
        alchol_df['class'][i] = j[0]

    if len(j) == 2 :
        if j[0] == '탁주' or j[0] == '과실주' or j[0] == '논알콜':
            alchol_df['class'][i] = j[0]
            alchol_df['category1'][i] = j[1]

        else :
            alchol_df['class'][i] = j[0] + " " +j[1]

    if len(j) == 3 :
        alchol_df['class'][i] = j[0] + " " + j[1]
        alchol_df['category1'][i] = j[2]


    if len(j) == 4 :
        alchol_df['class'][i] = j[0] + " " + j[1]
        alchol_df['category1'][i] = j[2] + " " + j[3]

    if len(j) == 5 :
        alchol_df['class'][i] = j[0] + " " + j[1]
        alchol_df['category1'][i] = j[2] + " " + j[3] + " " + j[4]



In [24]:
alchol_df.drop(columns= '580' , inplace= True)

In [35]:
alchol_df.to_csv('test.csv')

In [38]:
alchol_df.drop(columns= 'category', inplace= True)

In [41]:
alchol_df = alchol_df[['name', 'percent','class','category1','produce' ,'supply', 'img_url']]

In [44]:
alchol_df = alchol_df.rename(columns={"category1" : "category"})

In [45]:
alchol_df.to_csv('alchol_data.csv')